In [7]:
"""必要なモジュールの読み込み"""
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import cv2
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import clip



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)












cuda


In [8]:
# 画像にマスクをかける
from demo.simmim import MaskGenerator

# 画像の読み込み
image = cv2.imread("xrays/val_0.png")
image = cv2.resize(image, (512, 512))
## 前処理
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
image = transform(image)
mask = MaskGenerator(image)
# マスクの生成
masked_image = mask

# 画像の表示
## numpyに変換

# image Encoderの実装
from demo.image_encoder import ResnetEncoder

encoder = ResnetEncoder()
# imagetensorを(B, H, W, C)に変換
image = image.unsqueeze(0)

image = image.to(device)


encoder.to(device)
features = encoder.forward(image)
print(features.shape)










c:\Users\engineer\anaconda3\envs\ssl\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\engineer\anaconda3\envs\ssl\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 2048, 16, 16])


In [10]:
# 特徴量抽出された画像を復元
from demo.reconstruction import Recostruction

recostruction = Recostruction(encoder_outchannels=2048)

recostruction.to(device)

features = features.to(device)

reconstructed_image = recostruction.forward(features)

# 画像の表示
reconstructed_image = reconstructed_image.squeeze(0)
reconstructed_image = reconstructed_image.detach().cpu().numpy()
print(reconstructed_image.shape)

reconstructed_image = np.transpose(reconstructed_image, (1, 2, 0))
# 正規化を戻す

cv2.imwrite("reconstructed_image.png", reconstructed_image)






(3, 511, 511)


True

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate='none')
            (

In [10]:
import clip
import cv2
from PIL import Image


model, preprocess = clip.load("ViT-B/16", device=device)

image = Image.open("xrays/val_0.png")
image = preprocess(image).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)



    






    




torch.Size([1, 512])


In [3]:
from simmim import SwinWithMIM, MIMHead, MaskGenerator

maskgenerator = MaskGenerator()

# 画像の読み込みおよびマスキング
image = cv2.imread("xrays/val_0.png",)
image = cv2.resize(image, (1024, 1024))

# マスクの作成
mask = maskgenerator()
mask = torch.from_numpy(mask)
mask = mask.unsqueeze(2)
# 画像のマスキング
image = torch.from_numpy(image)

x_masked = image * (1- mask)

x_masked = x_masked.numpy()

cv2.imwrite( "masked_image.png", x_masked)

x_masked = torch.from_numpy(x_masked)







In [12]:
# モデルの読み込み
backbone = torchvision.models.swin_v2_t(pretrained=True)
swinmim_head = demo_MIMHead(backbone)

swinmim_head.to(device)
# 順伝播の開始
x_masked = x_masked.to(device)
swinmim_head.forward(x_masked)


demo_MIMHead(
  (backbone): SwinTransformer(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): Permute()
        (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): SwinTransformerBlockV2(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttentionV2(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (0): Linear(in_fea

RuntimeError: Input type (__int64) and bias type (float) should be the same